# Model Training - FastText

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import os 
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
import json

import warnings
warnings.filterwarnings("ignore")

In [12]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [13]:
data = pd.read_pickle("../"+params["model_pipeline"]["recipe_path"])

## Model Training

In [44]:
# fasttext_model = FastText(data.ingredients, **params["model_pipeline"]["fast_text"])
# fasttext_model.save("../" + params["model_pipeline"]["model_path"])
fasttext_model = FastText.load("../" + params["model_pipeline"]["model_path"])

In [18]:
# Quick sanity check
fasttext_model.wv.most_similar("chicken", topn=10)
len(fasttext_model.wv.index_to_key)

9416

### IDF (Inverse Document Frequency)

In [19]:
idf = {}
# Calculate IDF
for vocab in fasttext_model.wv.index_to_key:
    n_recipe_contains_vocab = data.ingredients.apply(lambda x: vocab in x).sum()
    idf[vocab] = np.log(len(data.ingredients) / (n_recipe_contains_vocab + 1e-9))

idf

{'salt': 0.43756323278446185,
 'garlic': 1.085610536358484,
 'water': 1.3433086441537483,
 'sugar': 1.5919917083637571,
 'soy_sauce': 1.6608262141840013,
 'black_pepper': 1.6979272435469352,
 'egg': 1.7484856093407433,
 'ginger': 1.874513430255987,
 'olive_oil': 1.9424564328800327,
 'vanilla_extract': 2.1806656847698176,
 'onion': 2.2918129960106466,
 'pepper': 2.370740254600195,
 'unsalted_butter': 2.3804490687271405,
 'lemon': 2.3984971385988136,
 'shaoxing_wine': 2.405141681317471,
 'baking_powder': 2.405141681317471,
 'milk': 2.4151920171709556,
 'cornstarch': 2.4424974678611764,
 'sesame_oil': 2.4599499178123723,
 'carrot': 2.5405796522123336,
 'maple_syrup': 2.5874632381110922,
 'scallion': 2.613807213450641,
 'vegetable_oil': 2.642975948041237,
 'butter': 2.66434428244689,
 'cinnamon': 2.6665064454513807,
 'white_pepper': 2.7085015664878913,
 'green_onion': 2.717571923457835,
 'brown_sugar': 2.7244290931839563,
 'all-purpose_flour': 2.7737928163021763,
 'baking_soda': 2.81310991

### Ingredient Vectors

In [20]:
# Prepare ingredient vectors
ingredient_vectors = []
for ingredients in data.ingredients:
    # Get the normalized vectors for the recipe ingredients and weight by IDF
    embedding_vecs = [fasttext_model.wv.get_vector(ing, norm=True) * idf.get(ing, 1) for ing in ingredients]
    ingredient_vectors.append(embedding_vecs)

ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in ingredient_vectors])

### Title Vectors

In [21]:
# Prepare title vectors
title_vectors = []
for title in data.recipe_title:
    # Get the vector for the recipe title and normalize it
    title_split = title.split()
    embedding_vecs = [fasttext_model.wv.get_vector(title, norm=True) for title in title_split]
    title_vectors.append(embedding_vecs)

title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in title_vectors])

### Query Processing

In [ ]:
# Query processing with IDF weighting
text = "rice, duck" # Example input
query = [ingredient.strip() for ingredient in text.split(",") if ingredient]
print("query:", query)

# Get the normalized vectors for the search query
query_vecs = np.array([fasttext_model.wv.get_vector(input, norm=True) for input in query])

# Borrow IDF from top 3 most similar ingredients
idf_top_weights = params["model_pipeline"]["model_scoring"]["idf_top_weights"] # Weights for top 3 similar ingredients

query_idfs = []
for query in query_vecs:
    similar_ings = [ing for ing, _ in fasttext_model.wv.similar_by_vector(query, topn=3)]
    ing_idf = np.sum([weight * idf.get(similar_ing, 1) for similar_ing, weight in zip(similar_ings, idf_top_weights)])
    query_idfs.append(ing_idf)

# Apply IDF 
query_vecs = np.array([(query_vec * query_idf) for query_vec, query_idf in zip(query_vecs, query_idfs)])

query: ['egg']


### Model Scoring System

In [49]:
# Prediction scoring system
# Calculate MaxSim for each recipe in the data
# Might be slow for large amount of data
ingredient_max_sim = np.array([])
for ing_vector in ingredient_vectors:
    tokens_sim = cosine_similarity(query_vecs, ing_vector)
    
    a_best = tokens_sim.max(axis=1)
    b_best = tokens_sim.max(axis=0)
    
    score = 0.5 * (a_best.mean() + b_best.mean())
    ingredient_max_sim = np.append(ingredient_max_sim, score)

# Calculate cosine similarity between the mean query vector and the mean ingredients vector
mean_query_vec = query_vecs.mean(axis=0)
ingredient_sim = cosine_similarity(mean_query_vec.reshape(1, -1), ingredient_vector_means)
title_sim = cosine_similarity(mean_query_vec.reshape(1, -1), title_vector_means)

score = (params["model_pipeline"]["model_scoring"]["w_cosine"] * ingredient_sim[0] + 
            params["model_pipeline"]["model_scoring"]["w_maxsim"] * ingredient_max_sim + 
            params["model_pipeline"]["model_scoring"]["w_title"] * title_sim[0])

rec_idx = np.argsort(score)[-params["model_service"]["n_recs"]:][::-1]
# rec_idx = [4557, 5297, 3977, 6109, 366, 3572, 6262, 314]
data.iloc[rec_idx]

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat,num_ingredients
1189,Ghirardelli Flourless Chocolate Cake,https://dailydishrecipes.com/ghirardelli-flour...,"[unsalted_butter, egg, semisweet_chocolate]",https://dailydishrecipes.com/wp-content/upload...,10.0,25.0,25.0,50.0,0.0,400.0,...,17.0,53.0,4.0,0.0,0.00,519.0,0.400,2.0,9.0,3
3006,Popovers With Strawberry Butter,https://www.justonecookbook.com/popovers-with-...,"[milk, all-purpose_flour, salt, baking_powder,...",https://www.justonecookbook.com/wp-content/upl...,13.0,15.0,45.0,120.0,0.0,319.0,...,7.0,124.0,2.0,60.0,1.00,487.0,1.000,1.0,4.0,8
1587,Ham Cheese Croissants,https://thewoksoflife.com/ham-cheese-croissants/,"[milk_bread_croissant_dough, ham, cheese, egg]",https://thewoksoflife.com/wp-content/uploads/2...,8.0,600.0,30.0,630.0,0.0,552.0,...,7.0,155.0,3.0,0.0,1.00,1147.0,1.000,2.0,10.0,4
2749,Matcha Basque Cheesecake,https://www.justonecookbook.com/matcha-basque-...,"[cream_cheese, sugar, egg, egg_yolk, matcha, c...",https://www.justonecookbook.com/wp-content/upl...,28.0,30.0,30.0,180.0,1.0,277.0,...,13.0,56.0,1.0,120.0,1.00,977.0,1.000,0.0,NaN,9
1066,Bacon Egg Muffin Cups,https://dailydishrecipes.com/bacon-egg-muffin-...,"[bread, egg, bacon, cheese]",https://dailydishrecipes.com/wp-content/upload...,7.0,10.0,10.0,20.0,0.0,207.0,...,1.0,67.0,1.0,0.0,0.03,278.0,0.050,3.0,6.0,4
5003,Brioche,https://www.recipetineats.com/brioche/,"[instant/rapid_rise_yeast, milk, plain/all_pur...",https://www.recipetineats.com/tachyon/2021/06/...,22.0,45.0,35.0,510.0,0.0,231.0,...,5.0,19.0,1.0,430.0,0.00,432.0,1.000,0.0,NaN,9
5644,Peanut Butter Chocolate Molten Lava Cake,https://www.recipetineats.com/peanut-butter-ch...,"[dark_chocolate_melt, unsalted_butter, egg, eg...",https://www.recipetineats.com/tachyon/2014/10/...,10.0,7.0,16.0,23.0,118.0,493.0,...,38.3,90.0,1.6,0.0,0.00,750.0,0.000,0.0,NaN,8
5590,Magic Three Layer Custard Cake,https://www.recipetineats.com/magic-3-layer-cu...,"[egg, sugar, vanilla_extract, unsalted_butter,...",https://www.recipetineats.com/tachyon/2015/04/...,14.0,20.0,50.0,70.0,114.0,247.0,...,19.4,80.0,0.9,0.0,0.00,400.0,0.000,0.0,NaN,9
4648,Super Soft Gluten-Free Sugar Cookies,https://minimalistbaker.com/super-soft-gluten-...,"[unsalted_butter, granulated_sugar, egg, vanil...",https://minimalistbaker.com/wp-content/uploads...,5.0,15.0,10.0,25.0,1.0,52.0,...,5.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,NaN,6
6432,The Buttermilk Pancakes,https://www.spoonforkbacon.com/the-best-butter...,"[purpose_flour, sugar, baking_powder, baking_s...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,5.0,7.0,12.0,0.0,458.0,...,12.0,192.0,2.0,0.0,0.00,911.0,1.000,1.0,7.0,9


## Wrap Model with MLFlow Python Model

Models from Code (Add set model) or Load as Class (Restart kernel after every overwrite)

In [24]:
class RecipeFastText(mlflow.pyfunc.PythonModel):
    def __init__(self, params):
        self.params = params

    def load_context(self, context):
        self.model = FastText.load(context.artifacts["model_path"])
        self.data = pd.read_pickle(context.artifacts["data_path"])

        # Prepare IDF
        self.idf = {}
        for vocab in self.model.wv.index_to_key:
            n_recipe_contains_vocab = self.data.ingredients.apply(lambda x: vocab in x).sum()
            self.idf[vocab] = np.log(len(self.data.ingredients) / (n_recipe_contains_vocab + 1e-9))

        # Prepare ingredient vectors
        self.ingredient_vectors = []
        for ingredients in self.data.ingredients:
            # Get the vectors for the recipe ingredients and normalize them
            embedding_vecs = [self.model.wv.get_vector(ing, norm=True) * self.idf.get(ing, 1)  for ing in ingredients]
            self.ingredient_vectors.append(embedding_vecs)

        self.ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in self.ingredient_vectors])

        # Prepare title vectors
        self.title_vectors = []
        for title in self.data.recipe_title:
            # Get the vector for the recipe title and normalize it
            title_split = title.split()
            embedding_vecs = [self.model.wv.get_vector(title, norm=True) for title in title_split]
            self.title_vectors.append(embedding_vecs)

        self.title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in self.title_vectors])

    def predict(self, model_input: list[str]) -> list[float]:
        """Predicts the recipe similarity score based on the input string."""
        
        # Get the vectors for the search query and normalize them
        query_vecs = np.array([self.model.wv.get_vector(input, norm=True) for input in model_input])

        # Borrow IDF from top 3 most similar ingredients
        idf_top_weights = self.params["model_scoring"]["idf_top_weights"] # Weights for top 3 similar ingredients

        query_idfs = []
        for query in query_vecs:
            similar_ings = [ing for ing, _ in self.model.wv.similar_by_vector(query, topn=3)]
            ing_idf = np.sum([weight * self.idf.get(similar_ing, 1) for similar_ing, weight in zip(similar_ings, idf_top_weights)])
            query_idfs.append(ing_idf)

        # Apply IDF 
        query_vecs = np.array([(query_vec * query_idf) for query_vec, query_idf in zip(query_vecs, query_idfs)])

        # Calculate MaxSim for each recipe in the data
        # Might be slow for large amount of data
        ingredient_max_sim = np.array([])
        for ing_vector in self.ingredient_vectors:
            tokens_sim = cosine_similarity(query_vecs, ing_vector)

            a_best = tokens_sim.max(axis=1)
            b_best = tokens_sim.max(axis=0)
            
            score = 0.5 * (a_best.mean() + b_best.mean())
            ingredient_max_sim = np.append(ingredient_max_sim, score)

        # Calculate cosine similarity between the mean query vector and the mean ingredients vector
        mean_query_vec = query_vecs.mean(axis=0)
        ingredient_sim = cosine_similarity(mean_query_vec.reshape(1, -1), self.ingredient_vector_means)
        title_sim = cosine_similarity(mean_query_vec.reshape(1, -1), self.title_vector_means)

        score = (self.params["model_scoring"]["w_cosine"] * ingredient_sim[0] +
                self.params["model_scoring"]["w_maxsim"] * ingredient_max_sim +
                self.params["model_scoring"]["w_title"] * title_sim[0])

        return score

### Log Model to MLFlow

In [25]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("FastText-Model")

<Experiment: artifact_location='mlflow-artifacts:/395968226849530716', creation_time=1758176343047, experiment_id='395968226849530716', last_update_time=1758176343047, lifecycle_stage='active', name='FastText-Model', tags={}>

In [31]:
# Load params
params = load_params()

with mlflow.start_run():
    # Load data
    data = pd.read_pickle("../" + params["model_pipeline"]["recipe_path"])

    # Load fasttext model
    fasttext_model = RecipeFastText(params["model_pipeline"])

    # Log params
    mlflow.log_params(params["model_pipeline"]["fast_text"])
    mlflow.log_params(params["model_pipeline"]["model_scoring"])

    # Log model
    model_info = mlflow.pyfunc.log_model(
        name="fasttext_model",
        python_model=fasttext_model,
        artifacts={"model_path": "../" + params["model_pipeline"]["model_path"],
                   "data_path": "../" + params["model_pipeline"]["recipe_path"]},
        pip_requirements=["gensim==4.3.3"]
    )

    # Log local URI
    local_uri = os.path.join("../mlflow/mlartifacts", *model_info.artifact_path.split("/")[1:])
    mlflow.log_params({"local_uri": local_uri})
    
    # Log metadata
    metadata = {
        "server_uri": model_info.model_uri,
        "local_uri": local_uri,
        "run_id": model_info.run_id,
        "time_logged": model_info.utc_time_created,
        "model_params": params["model_pipeline"]
    }

    # Save metadata locally
    metadata["model_params"] = params["model_pipeline"]
    with open("../training_metadata.json", "w") as f:
        f.write(json.dumps(metadata, indent=4))

    mlflow.log_artifact("../training_metadata.json")


🏃 View run unequaled-auk-279 at: http://127.0.0.1:8080/#/experiments/395968226849530716/runs/b51bd046c0d5455d872f746bf44c0d06
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/395968226849530716


### Quick Sanity Check

In [ ]:
with open("../training_metadata.json", "r") as f:
    metadata = json.load(f)

model = mlflow.pyfunc.load_model(metadata["local_uri"])

In [33]:
sim_scores = model.predict(["bread"])  # Example usage
rec_idx = np.argsort(sim_scores)[::-1][:10] # Get top recommendations
data.iloc[rec_idx]

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat,num_ingredients
3488,Breadcrumbs,https://minimalistbaker.com/how-to-make-breadc...,"[torn/cubed_bread, italian_herb_seasoning, gar...",https://minimalistbaker.com/wp-content/uploads...,7.0,5.0,20.0,25.0,1.0,114.0,...,2.4,62.0,1.5,0.0,0.0,0.0,0.00,0.7,0.3,4
5167,Grilled Garlic Bread,https://www.recipetineats.com/grilled-garlic-b...,"[bread_loaf, /_stick_salted_butter, garlic, pa...",https://www.recipetineats.com/tachyon/2019/06/...,9.0,5.0,10.0,15.0,0.0,331.0,...,4.0,100.0,2.4,0.0,1.3,575.0,0.00,0.0,NaN,4
5250,Christmas Leftovers Bread Bowl Sandwich,https://www.recipetineats.com/bread-bowl-sandw...,"[round/oval_bread_loaf, ham_turkey, meat_slice...",https://www.recipetineats.com/tachyon/2018/12/...,10.0,15.0,30.0,45.0,0.0,574.0,...,2.0,226.0,4.4,0.0,5.4,2160.0,0.00,0.0,NaN,8
5813,Garlic Bread,https://www.loveandlemons.com/garlic-bread/,"[unsalted_butter, parsley/chives, garlic, salt...",https://cdn.loveandlemons.com/wp-content/uploa...,4.0,10.0,10.0,20.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,NaN,8
5232,Quick Cheesy Garlic Bread,https://www.recipetineats.com/quick-cheesy-gar...,"[bread_loaf_approx_/″, unsalted_butter, salt, ...",https://www.recipetineats.com/tachyon/2019/01/...,7.0,5.0,25.0,30.0,0.0,267.0,...,0.0,117.0,1.5,0.0,0.2,400.0,0.00,0.0,NaN,6
683,Melted Brie And Bacon On Grilled Flatbread,https://dailydishrecipes.com/melted-brie-bacon...,"[bread, brie_cheese, bacon, rom_tomato, season...",https://dailydishrecipes.com/wp-content/upload...,3.0,5.0,7.0,12.0,0.0,478.0,...,2.0,164.0,1.0,0.0,8.0,1030.0,0.10,4.0,14.0,6
5670,No Washing Up Ham Egg Cheese Bread Bowls,https://www.recipetineats.com/no-washing-up-ha...,"[bread_roll, slice_ham, egg, mozzarella_cheese...",https://www.recipetineats.com/tachyon/2014/09/...,8.0,5.0,15.0,20.0,194.0,415.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,NaN,5
6368,Thanksgiving Sandwich,https://www.spoonforkbacon.com/thanksgiving-tu...,"[bread, mayonnaise, cranberry_sauce, roasted_t...",https://www.spoonforkbacon.com/wp-content/uplo...,4.0,10.0,0.0,10.0,0.0,1030.0,...,63.0,179.0,6.0,0.0,6.0,2607.0,1.00,18.0,11.0,7
1145,Ciabatta Bread Bacon Pizza,https://dailydishrecipes.com/ciabatta-bread-ba...,"[ciabatta_bread, mozzarella, basil, garlic, ol...",https://dailydishrecipes.com/wp-content/upload...,8.0,5.0,10.0,17.0,0.0,425.0,...,1.0,296.0,0.5,2.0,1.0,550.0,0.03,2.0,10.0,6
672,Brie And Raspberry Panini With Hazelnut Spread,https://dailydishrecipes.com/brie-and-raspberr...,"[sourdough_bread, butter, chocolate-hazelnut_s...",https://dailydishrecipes.com/wp-content/upload...,2.0,5.0,5.0,10.0,0.0,598.0,...,17.0,143.0,6.0,0.0,2.0,346.0,0.20,2.0,4.0,5
